In [2]:
import numpy as np
from huggingface_utils.metrics.ner import NerMetric

def fake_labels_and_logits(N, T, K):
    labels = np.random.randint(0,K, size = (N,T))
    labels[:,0] = -100
    labels[:,-1] = -100
    labels[-1,-2:] = -100
    labels[-2,-4:] = -100
    logits = np.random.random((N,T,K))
    return labels, logits

label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
labels, logits = fake_labels_and_logits(100, 20, len(label_names))


compute_metrics = NerMetric(label_names)
print(compute_metrics((logits, labels)))
    

{'precision': 0.16445993031358885, 'recall': 0.16043507817811012, 'f1': 0.16242257398485893, 'accuracy': 0.111358574610245}
